# One-shot face recognition tutorial

This tutorial based on this [video](https://www.youtube.com/watch?v=LKispFFQ5GU) will use tensorflow v2 to create a face recognition model based on the one-shot architecture. <br/>

The steps are:

## Workspace setup

Let's import all the shared libraries that notebook uses section-wide

In [1]:
from os import path
from typing import Final, Any

from tqdm.auto import tqdm
import matplotlib.pyplot as plt

c:\Users\user\Desktop\Noam\one-shot-fr-tutorial\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's create a workspace with all the necessary folders in it

In [2]:
import os

WORKSPACE_DIR_PATH: Final[str] = 'tutorial_workspace'
DOWNLOADS_PATH: Final[str] = path.join(WORKSPACE_DIR_PATH, 'downloads')
ANCHORS_PATH: Final[str] = path.join(WORKSPACE_DIR_PATH, 'anchors')
POSITIVES_PATH: Final[str] = path.join(WORKSPACE_DIR_PATH, 'positives')
NEGATIVES_PATH: Final[str] = path.join(WORKSPACE_DIR_PATH, 'negatives')

os.makedirs(DOWNLOADS_PATH, exist_ok=True)
os.makedirs(ANCHORS_PATH, exist_ok=True)
os.makedirs(POSITIVES_PATH, exist_ok=True)
os.makedirs(NEGATIVES_PATH, exist_ok=True)

print('All necessary folders created')

All necessary folders created


## Data fetching

To train a model a dataset is necessary. <br/>
For that we will use the `LFW` dataset

### Fetch dataset

The dataset we will use named `LFW` and you can find it either in this [link](http://vis-www.cs.umass.edu/lfw/#download) or at the code below

In [3]:
import shutil
import requests

TAR_URL: Final[str] = 'http://vis-www.cs.umass.edu/lfw/lfw.tgz'
FILE_NAME: Final[str] = 'lfw.tgz'
file_path: str = path.join(DOWNLOADS_PATH, FILE_NAME)

if not path.exists(file_path):
    # Make an HTTP request within a context manager
    with requests.get(TAR_URL, stream=True) as r:
        
        # Check header to get content length, in bytes
        total_length = int(r.headers.get("Content-Length"))
        
        # Implement progress bar via tqdm
        with tqdm.wrapattr(r.raw, "read", total=total_length, desc="lfw.tar compressed dataset") as raw:
        
            # Save the output to a file
            with open(file_path, 'wb')as output:
                shutil.copyfileobj(raw, output)
else:
    print('Dataset already downloaded')

Dataset already downloaded


### Extract the dataset

As described above, the model needs a lot faces that not match to the anchors. <br/>
For that we will use the dataset we downloaded above.

So, we will extract the compressed files. and put them into the `NEGATIVES_PATH`.

In [6]:
import tarfile
from typing import IO, Iterable

file_path: str = path.join(DOWNLOADS_PATH, 'lfw.tgz')

with tarfile.open(file_path, 'r:gz') as compressed_file:
    # Filtering out all directories and non-jpg files
    all_members: Iterable[tarfile.TarInfo] = [m for m in compressed_file.getmembers() if '.jpg' in m.name or '.png' in m.name]
    
    for member in tqdm(iterable=all_members, total=len(all_members)):
        archive_filename: str = member.path.split('/')[-1]
        filename: str = path.join(NEGATIVES_PATH, archive_filename)
        
        buffer_reader: IO[bytes] = compressed_file.extractfile(member)
        with open(filename, 'wb') as file:
            file.write(buffer_reader.read())

100%|██████████| 13233/13233 [01:22<00:00, 159.45it/s]


### Collect positive and anchors data

Until now, we successfully loaded the `LFW` dataset into our workspace as negative shots (observations).

Now, let's dive into the positives.
So, we can get those observations in many ways. Like taking shots from the camera, use existing images. <br/>
Let's do both!

#### Taking shots from webcam

Before we take shots from the camera, we need to remember. The dataset has 250x250 pixels images. <br/>
For the simplicity of this project, we will preffer to use the exact same size with the new shots.

So remember, 250x250...

Don't forget to import necessary packages for that phase

In [3]:
import cv2
import uuid

In [4]:
# In case it not working properly, you can try with different index
cap = cv2.VideoCapture(0)
is_device_connected: bool = cap.isOpened()
print(f"The device is { 'connected' if is_device_connected else 'disconnected' }")
while is_device_connected:
    ret, frame = cap.read()

    # Cut a 250x250 pixels crop from the original feed
    start_x: int = 200
    start_y: int = 120
    height: int = start_y + 250
    width: int = start_x + 250
    frame = frame[start_x:width, start_y:height, :]

    # Collect anchor image
    if cv2.waitKey(1) & 0XFF == ord('a'):
        # Creating unique filename
        generated_name: str = f'{uuid.uuid1()}.jpg'
        img_name: str = path.join(ANCHORS_PATH, generated_name)
        cv2.imwrite(img_name, frame)

    # Collect positive image
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Creating unique filename
        generated_name: str = f'{uuid.uuid1()}.jpg'
        img_name: str = path.join(POSITIVES_PATH, generated_name)
        cv2.imwrite(img_name, frame)

    # Show image back to screen
    cv2.imshow('Image collection', frame)

    if cv2.waitKey(1) & 0XFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

The device is connected


In [ ]:
print('Frame shape:', frame.shape)
plt.title('Last frame detected')
plt.imshow(frame)

##### In case the code above not working

In case you don't know which `device id` to use in the opencv's video `device id`. <br />
Let's search for all the devices available to take images from

In [3]:
def list_ports():
    """
        Test the ports and returns a tuple with the available ports and the ones that are working.
    """
    non_working_ports = []
    dev_port = 0
    working_ports = []
    available_ports = []
    while len(non_working_ports) < 6: # if there are more than 5 non working ports stop the testing. 
        print(f'Testing index {dev_port}')
        camera = cv2.VideoCapture(dev_port)
        if not camera.isOpened():
            non_working_ports.append(dev_port)
            print("Port %s is not working." %dev_port)
        else:
            is_reading, img = camera.read()
            w = camera.get(3)
            h = camera.get(4)
            if is_reading:
                print("Port %s is working and reads images (%s x %s)" %(dev_port,h,w))
                working_ports.append(dev_port)
            else:
                print("Port %s for camera ( %s x %s) is present but does not reads." %(dev_port,h,w))
                available_ports.append(dev_port)
        dev_port +=1
    return available_ports,working_ports,non_working_ports

print(list_ports())

Testing index 0
Port 0 is working and reads images (480.0 x 640.0)
Testing index 1
Port 1 is not working.
Testing index 2
Port 2 is not working.
Testing index 3
Port 3 is not working.
Testing index 4
Port 4 is not working.
Testing index 5
Port 5 is not working.
Testing index 6
Port 6 is not working.
([], [0], [1, 2, 3, 4, 5, 6])


#### Load data from existing images

Alternitivly, you can load the positive and anchors images from existing images files. <br/>
Just copy your images positive images into `$POSITIVES_PATH`, and the anchors will be copied into `ANCHORS_PATH`.

Don't forget to make sure that the photos you are importing have to be at shape (resolution) of `250px*250px`

## Preprocess

All the data successfully collected. Let's start to prepare it to the training phase.

So, what we will do in this phase is:
* Create labeled dataset from the anchors, positives and negatives directories
* Split the dataset into training and validation sets

### Phase imports

In [28]:
from typing import Union
import tensorflow as tf
from tf_agents.typing.types import EagerTensor

### Data loading

Let's verify that all of the images we will going to use, having the same 'shape' (dimensions). <br/>
In case they are different, we will scale them to the apropriate dimensions.

In [29]:
DATASET_SIZE: Final[int] = 300

# Tensorflow will take all files matching to the pattern inside `list_files()`
anchor: tf.data.Dataset = tf.data.Dataset.list_files(f'{ANCHORS_PATH}/*.jpg').take(DATASET_SIZE)
positive: tf.data.Dataset = tf.data.Dataset.list_files(f'{POSITIVES_PATH}/*.jpg').take(DATASET_SIZE)
negative: tf.data.Dataset = tf.data.Dataset.list_files(f'{NEGATIVES_PATH}/*.jpg').take(DATASET_SIZE)

negative.as_numpy_iterator(), anchor.as_numpy_iterator(), positive.as_numpy_iterator()

(<tensorflow.python.data.ops.dataset_ops._NumpyIterator at 0x1d673ea9660>,
 <tensorflow.python.data.ops.dataset_ops._NumpyIterator at 0x1d673eaaa70>)

Example: This how you can iterate over a tensorlow Dataset class (in this case we will run over the anchors)

In [30]:
anchor.as_numpy_iterator().next()

b'tutorial_workspace\\anchors\\de5caf61-9cb0-11ed-90af-b808cf4c5169.jpg'

### Scale and resize the images

In [31]:
def preprocess(file_path: str):
    # Read image bytes from file path
    byte_image = tf.io.read_file(file_path)
    
    # Loading the bytes as image
    image = tf.io.decode_jpeg(byte_image)

    # Resize the image to 100x100 pixels
    image: EagerTensor = tf.image.resize(image, (100, 100))
    
    # Devide each pixel between (0 and 1) instead of (0 and 255)
    image /= 255.0

    return image

Just to be sure, let's verify that the pre process have done successfully.

In [ ]:
# Duplicate one of the lists 
_dup_anchor = anchor.as_numpy_iterator()

# Running the preprocess on the current image
img = preprocess(_dup_anchor.next())

print('Minimum value in tensor (should be 0):', img.numpy().min())
print('Max value in tensor (should be 1):', img.numpy().max())
print('Image shape (should be (100, 100, 3)):', img.numpy().shape)
plt.imshow(img)

### Create the labeled dataset

To train the model, we will need to give it:
* Positive observations (which means anchor image, positive image and result of 1).
* Negative observations (which means anchor image, negative and result of 0).

For that we will create two datasets, one for positive and one for negative. And concatenate them together.

In [34]:
positive_labels = tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor))) # Create a vector with shape of positive images
negative_labels = tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor))) # Create a vector with shape of negative images

# Telling the data loader to load each of those images with the proper label simultaneously 
positive_dataset = tf.data.Dataset.zip((anchor, positive, positive_labels))
negative_dataset = tf.data.Dataset.zip((anchor, negative, negative_labels))

# Concatenating the positive and negative datasets into a single dataset
dataset = positive_dataset.concatenate(negative_dataset)

#### Little example

In [35]:
sample = dataset.as_numpy_iterator()
dataset

<ConcatenateDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>

In [36]:
sample.next()

(b'tutorial_workspace\\anchors\\d9f50c46-9cb0-11ed-925b-b808cf4c5169.jpg',
 b'tutorial_workspace\\positives\\d99051b8-9cb0-11ed-a54b-b808cf4c5169.jpg',
 1.0)

### Build train and test dataset partitions

Let's load all the images from it's path, and put them the proper label

In [37]:
def preprocess_twin(input_img: str, validation_img: str, label: int) -> tuple[Any, Any, int]:
    return (preprocess(input_img), preprocess(validation_img), label)

Let's test this function

In [38]:
example = sample.next()
res = preprocess_twin(*example)
res

(<tf.Tensor: shape=(100, 100, 3), dtype=float32, numpy=
 array([[[0.57254905, 0.60784316, 0.6039216 ],
         [0.5742647 , 0.6095588 , 0.60563725],
         [0.5735294 , 0.60882354, 0.60490197],
         ...,
         [0.61960787, 0.6627451 , 0.64705884],
         [0.622549  , 0.66568625, 0.65      ],
         [0.622549  , 0.66568625, 0.65      ]],
 
        [[0.5732843 , 0.60857844, 0.6046569 ],
         [0.56789213, 0.60318625, 0.5992647 ],
         [0.56960785, 0.60784316, 0.6039216 ],
         ...,
         [0.6193628 , 0.6634804 , 0.6477941 ],
         [0.6186274 , 0.6627451 , 0.64705884],
         [0.6186274 , 0.6627451 , 0.64705884]],
 
        [[0.5688726 , 0.6041667 , 0.6002451 ],
         [0.5627451 , 0.6009804 , 0.59705883],
         [0.5688726 , 0.60784316, 0.6039216 ],
         ...,
         [0.6159314 , 0.6629902 , 0.64730394],
         [0.61960787, 0.6666667 , 0.6509804 ],
         [0.6166667 , 0.6637255 , 0.6480392 ]],
 
        ...,
 
        [[0.0875    , 0.07965686

#### Example

So, after we have tested everything. Let's put all the things together. And build our dataloader pipline

In [39]:
dataset = dataset.map(preprocess_twin) # Running all over the dataset with the `preprocess_twin` function
dataset = dataset.cache() # Caching the images
dataset = dataset.shuffle(buffer_size=1024) # Shuffling the dataset
dataset

<ShuffleDataset element_spec=(TensorSpec(shape=(100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>

In [ ]:
samp = dataset.as_numpy_iterator().next()

print('Label is:', samp[2])
plt.imshow(samp[0])

In [ ]:
plt.imshow(samp[1])

#### Make training and test dataset partitions

Shared variables for this section

In [42]:
LEARNING_BATCH_SIZE: Final[int] = 16
LEARNING_PREFETCH_SIZE: Final[int] = 8

TRAIN_PARTITION_SIZE: Final[int] = round(len(dataset) * .7) # Get 70% of the dataset for training
TEST_PARTITION_SIZE: Final[int] = round(len(dataset) * .3) # Get 30% of the dataset for testing

The training partition setup:

In [43]:
training_data: tf.data.Dataset = dataset.take(TRAIN_PARTITION_SIZE)
training_data: tf.data.Dataset = training_data.batch(LEARNING_BATCH_SIZE) # Set batch size of 16
training_data: tf.data.Dataset = training_data.prefetch(LEARNING_PREFETCH_SIZE)
training_data

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(None, 100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

Setup testing partition

In [44]:
test_data: tf.data.Dataset = dataset.skip(TRAIN_PARTITION_SIZE) # Skipping of all the training partition data
test_data: tf.data.Dataset = test_data.take(TEST_PARTITION_SIZE)
test_data: tf.data.Dataset = test_data.batch(LEARNING_BATCH_SIZE)
test_data: tf.data.Dataset = test_data.prefetch(LEARNING_PREFETCH_SIZE)
test_data

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(None, 100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

## Model engineering

### Shared imports for this section

In [52]:
from keras.models import Model
from keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten

In [46]:
from typing import Tuple

# 100x100 px, with 3 colors channels (RGB)
INPUT_IMAGE_SHAPE: Final[Tuple] = (100, 100, 3)

### GPU memory limit

In case you have a GPU installed on your machine. It's recommended to set a memory limit

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

### Build the embedding layer

In [53]:
def make_embedding() -> Model:
    # Creating the input layer
    input_layer = Input(shape=INPUT_IMAGE_SHAPE, name='input_image')

    # First block
    c1 = Conv2D(64, (10, 10), activation='relu')(input_layer)
    m1 = MaxPooling2D(64, (2, 2), padding='same')(c1)

    # Second bloc
    c2 = Conv2D(128, (7, 7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2, 2), padding='same')(c2)

    # Third block
    c3 = Conv2D(128, (4, 4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2, 2), padding='same')(c3)

    # Final embedding block
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)

    return Model(inputs=[input_layer], outputs=[d1], name='embedding')

So, let's try to compile it

In [54]:
# Creating the embedding model/layer and showing the summary of it
embedding: Model = make_embedding()
embedding.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 100, 100, 3)]     0         
                                                                 
 conv2d_8 (Conv2D)           (None, 91, 91, 64)        19264     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 46, 46, 64)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 40, 40, 128)       401536    
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 20, 20, 128)      0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 17, 17, 128)       26

### Build Distance Layer

TODO: Need to check if arcface will perform better than this

In [55]:
# Siamese L1 Distance class
class L1Dist(Layer):
    
    # Init method - inheritance
    def __init__(self, **kwargs) -> None:
        super().__init__()

    # Smiliarity calculation
    def call(self, input_embeddings, validation_embeddings) -> Any:
        return tf.math.abs(input_embeddings - validation_embeddings)

### Making Siamese Model

In [56]:
def make_siamese_model() -> Model:
    # Anchor image input in the network
    input_image = Input(name='input_image', shape=INPUT_IMAGE_SHAPE)

    # Validation image in the network
    validation_image = Input(name='validation_image', shape=INPUT_IMAGE_SHAPE)

    # Combine siamese distance components
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))

    # Classification layer
    classifier = Dense(1, activation='sigmoid')(distances)
    return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

Compilation of this network

In [57]:
# Creating the embedding model/layer and showing the summary of it
siamese_model: Model = make_siamese_model()
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 validation_image (InputLayer)  [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4096)         38960448    ['input_image[0][0]',            
                                                                  'validation_image[0

## Training the model

The steps we will follow:
1. Setup a loss function
2. Setup an optimizer
3. Build a Custom training step
4. Create a Training loop
5. Train the model